<a href="https://colab.research.google.com/github/bilalProgTech/btech-nmims/blob/master/NN-DL/Lab-Work/20230806-Lab-3-BTech-NNDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content'
!kaggle datasets download -d kausthubkannan/5-flower-types-classification-dataset

In [ ]:
!unzip *.zip

# CNN - Augmentation and Model

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
import os
image_path_list = []
for dirname, _, filenames in os.walk('/content/flower_images'):
    for filename in filenames:
        image_path_list.append(os.path.join(dirname, filename))

In [ ]:
train_df = pd.DataFrame({'filename': image_path_list})
train_df['class'] = train_df['filename'].str.split('/').str[-2]
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df['class'].value_counts()

In [ ]:
unique_feature = train_df['class'].unique()
len(unique_feature)

In [ ]:
sample = train_df.sample(16)
files = sample['filename'].tolist()
targets = sample['class'].tolist()

In [ ]:
files

In [ ]:
nrows = 4
ncols = 4
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

for i, (img_path, target) in enumerate(zip(files, targets)):
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off')

    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.title(target)

plt.show()

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    target_size=(256, 256),
    class_mode='categorical')

In [ ]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=(256, 256, 3)),
                             tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(len(unique_feature), activation='softmax')])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=10)

In [ ]:
model.evaluate(train_generator)

In [ ]:
successive_outputs = [layer.output for layer in model.layers]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

img = tf.keras.preprocessing.image.load_img(files[1], target_size=(256, 256))
x = tf.keras.preprocessing.image.img_to_array(img)
print(targets[1])
print(x.shape)
x = x.reshape((1,) + x.shape)
print(x.shape)
x /= 255
successive_feature_maps = visualization_model.predict(x)
layer_names = [layer.name for layer in model.layers]
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    if len(feature_map.shape) == 4:
        n_features = feature_map.shape[-1]
        print(feature_map.shape)
        size = feature_map.shape[1]
        display_grid = np.zeros((size, size * n_features))
        for i in range(n_features):
            x = feature_map[0, :, :, i]
            x -= x.mean()
            x /= x.std()
            x *= 64
            x += 128
            x = np.clip(x, 0, 255).astype('uint8')
            display_grid[:, i * size : (i + 1) * size] = x
        scale = 20. / n_features
        plt.figure(figsize=(scale * n_features, scale))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                          rotation_range=40,
                                                          width_shift_range=0.4,
                                                          height_shift_range=0.4,
                                                          shear_range=0.2,
                                                          zoom_range=0.2,
                                                          horizontal_flip=True,
                                                          validation_split=0.2)


train_generator = datagen.flow_from_directory(
    '/content/flower_images',
    class_mode = 'categorical',
    target_size=(256,256),
    subset = 'training')

val_generator = datagen.flow_from_directory(
    '/content/flower_images',
    class_mode = 'categorical',
    target_size=(256,256),
    subset = 'validation')

In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

# Auto Encoder - Image Compression

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_generator = datagen.flow_from_directory(
    '/content/flower_images',
    target_size=(256, 256),
    color_mode='rgb',
    shuffle=True,
    seed=42,
    class_mode='input')

In [ ]:
tf.keras.backend.clear_session()
class AutoEncoder(tf.keras.models.Model):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = tf.keras.Sequential([
        ])

        self.decoder = tf.keras.Sequential([
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = AutoEncoder()

In [ ]:
autoencoder.encoder.summary()

In [ ]:
autoencoder.decoder.summary()

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
%%time
history = autoencoder.fit(train_generator, epochs=10)

In [ ]:
num_images = len(files)
fig, axes = plt.subplots(num_images, 2, figsize=(8, 4*num_images))

for i, image_path in enumerate(files):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = np.vstack([x])
    x = x / 255
    encoded = autoencoder.encoder(x).numpy()
    decoded = autoencoder.decoder(encoded).numpy().reshape(-1, 256, 256, 3)

    axes[i, 0].imshow(x[0])
    axes[i, 0].set_title('Original')

    axes[i, 1].imshow(decoded[0])
    axes[i, 1].set_title('Decoded')

plt.show()

In [ ]:
img = tf.keras.preprocessing.image.load_img('',
                                            target_size=(256, 256))
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = np.vstack([x])
x = x / 255

encoded = autoencoder.encoder(x).numpy()
decoded = autoencoder.decoder(encoded).numpy().reshape(-1, 256, 256, 3)

plt.subplot(1, 2, 1)
plt.imshow(x[0])
plt.title('Original')

plt.subplot(1, 2, 2)
plt.imshow(decoded[0])
plt.title('Decoded')

plt.show()

img = tf.keras.preprocessing.image.array_to_img(decoded[0])
img.save('decoded_image.jpg')

In [ ]:
tf.keras.backend.clear_session()
class AutoEncoder(tf.keras.models.Model):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = tf.keras.Sequential([
        ])

        self.decoder = tf.keras.Sequential([
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = AutoEncoder()